In [ ]:
import pandas as pd
import numpy as np
import gensim
import nltk
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(1960)

nltk.download('wordnet')

In [ ]:
movies1_df = pd.read_json('1960_1964.json')
movies2_df = pd.read_json('1965_1970.json')
movies_df = movies1_df.append(movies2_df)
print(movies_df.shape)
movies_df.head()

In [ ]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer('english')
    return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
doc_sample = movies_df[movies_df['title'] == 'Bombs on Monte Carlo (1960 film)'].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

In [ ]:
processed_docs = movies_df['summary'].map(preprocess)


In [ ]:
dictionary  = gensim.corpora.Dictionary(processed_docs)

count = 0
for k, v in dictionary.iteritems():
    count += 1
    if count > 10:
        break

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [ ]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]


In [ ]:
lda_model = gensim.models.LdaMulticore(corpus_tfidf,
                                      num_topics=40,
                                      id2word=dictionary, 
                                      passes=2,
                                      workers=4)

for idx, topic in lda_model.print_topics(-1):
    print('Topic : {} Word: {}'.format(idx, topic))